In [1]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [2]:
import requests
import time
import os
import sys
import re
import io
import json
import csv
import openpyxl
import pandas as pd
import numpy as np
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import kss

In [3]:
#iframe 제거 후 blog.naver.com 붙이기
def delete_iframe(url):
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    src_url = "https://blog.naver.com/" + soup.iframe["src"]
    
    return src_url


# 본문 스크래핑
def text_scraping(url):
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    if soup.find("div", attrs={"class":"se-main-container"}):
        text = soup.find("div", attrs={"class":"se-main-container"}).get_text()
        text = text.replace("\n","")
        return text

    elif soup.find("div", attrs={"id":"postViewArea"}):
        text = soup.find("div", attrs={"id":"postViewArea"}).get_text()
        text = text.replace("\n","") 
        return text
    else:
        return "확인불가"
    
    
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break  

In [4]:
import datetime
from urllib.parse import quote

browser = Chrome()

query = "댕댕이" # 키워드 입력

url = "https://search.naver.com/search.naver?where=view&sm=tab_jum&query=" + quote(query)

browser.get(url)

doScrollDown(10)
    
soup = BeautifulSoup(browser.page_source, 'html.parser')    
posts = soup.find_all("li", attrs={"class":"bx _svp_item"})


for post in posts:
    post_title = post.find("a", attrs={"class":"api_txt_lines total_tit _cross_trigger"}).get_text()

    post_link = post.find("a", attrs={"class":"api_txt_lines total_tit _cross_trigger"})['href']


In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import quote


for post in posts:
    post_title = post.find("a", attrs={"class":"api_txt_lines total_tit _cross_trigger"}).get_text()
    post_link = post.find("a", attrs={"class":"api_txt_lines total_tit _cross_trigger"})['href']
    
    blog_p = re.compile("blog.naver.com")
    blog_m = blog_p.search(post_link)
    
    if blog_m:
        blog_text = text_scraping(delete_iframe(post_link))
        
        blog_t=(re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', blog_text))
        blog_t=(re.sub('[\xa0|0xed]', '', blog_t))
        blog_t=(re.sub('[a-zA-Z0-9一-龥]', '', blog_t))
        #blog_t=(re.sub('[-=+,#/\?:^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·★•♥❤️♪♬♡▪%♠▶]', ' ', blog_t))
        blog_t=(re.sub('[-=+,#/\?:^@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', blog_t))
        blog_t=(re.sub('[\U00010000-\U0010FFFF"]', '', blog_t))
            
        f = open('naver_data.txt', 'a',encoding='utf8')
        f.write(post_title + '\n' + blog_t + '\n')
        f.close()
        
        #print(post_title + '\n' + blog_t + '\n')
        #print("-"*50)
        
#browser.close()   

In [ ]:
browser.close()   

In [3]:
import os
import re
import hgtk # 자음 모음 관리
import pandas as pd
from konlpy.tag import Mecab, Okt

compose_code = "\u33c6"

def _jamo_sentence(sent):
    doublespace_pattern = re.compile(r"\s+")

    def __transform(char):
        if char == " ":
            return char
        cjj = hgtk.text.decompose(char, compose_code)
        if not cjj:
            return char
        if len(cjj) == 1:
            return cjj
        cjj_ = "".join(c if c != " " else compose_code for c in cjj)
        return cjj_

    sent_ = "".join(__transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(" ", sent_) # 두칸 이상 공백을 1칸 공백으로
    return sent_


def trans(char):
    if char == " ":
        return char
    cjj = hgtk.text.decompose(char, compose_code)
    if not cjj:
        return char
    if len(cjj) == 1:
        return cjj
    cjj_ = "".join(c if c != " " else compose_code for c in cjj)
    return cjj_

In [4]:
with open('naver_data.txt','r', encoding='utf-8') as f:
    datafile = f.readlines()
    datafile = ''.join(datafile)
    datafile=(re.sub(r'[^ ㄱ-ㅣ가-힣]', '', datafile))
    datafile=(re.sub(r'[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', datafile))

    datafile=_jamo_sentence(datafile)
    #print(datafile)
    
    f = open('naver_re_data.txt', 'a',encoding='utf8')
    f.write(datafile)
    f.close()

In [5]:
from konlpy.tag import Okt

okt = Okt() 
data = []

with open('naver_data.txt','r', encoding='utf-8') as f:
    datafile = f.readlines()
    datafile = ''.join(datafile)
    datafile=(re.sub(r'[^ ㄱ-ㅣ가-힣]', '', datafile))
    data.append(okt.nouns(datafile))
    #print(data)
    
    t = open('naver_noun.txt', 'w', encoding='utf-8')
    for i in data:
        t.write(str(i))
        t.close()

# 형태 분석

In [8]:
import datetime
import time
import fasttext

save = 'naver_data.txt'
model_fname = 'data_result.bin'

print("학습 중")
model1 = fasttext.train_unsupervised(
    input=save,
    model='cbow',
    loss='hs',
    ws=11,
    lr=0.0001,
    dim=10,
    epoch=1000,
    min_count=5,
    thread=12,
    verbose=1,
)

model1.save_model(model_fname)
print('완료')

학습 중
완료


In [9]:
#문장 학습 - 형태 유사어 추출
model1 = fasttext.load_model('data_result.bin')
model1.get_nearest_neighbors("댕댕이")

[(0.9838256239891052, '사실'),
 (0.9823348522186279, '강아지를'),
 (0.9820657968521118, '댕댕이는'),
 (0.9797086715698242, '댕댕이들'),
 (0.9775006175041199, '경우'),
 (0.9774555563926697, '강아지와'),
 (0.9769712686538696, '댕댕이가'),
 (0.9754970669746399, '강아지는'),
 (0.9742645025253296, '강아지'),
 (0.9709970355033875, '있는')]

## 명사 형태로 이미 전처리 후 하는 것은 의미 없음

In [12]:
save2 = 'naver_noun.txt'
model_fname2 = 'noun_result.bin'

print("학습 중")
model2 = fasttext.train_unsupervised(
    input=save2,
    model='cbow',
    loss='hs',
    ws=11,
    lr=0.0001,
    dim=50,
    epoch=100,
    min_count=5,
    thread=12,
    verbose=1,
)

model2.save_model(model_fname2)
print('완료')

학습 중
완료


In [13]:
#명사 학습 - 형태 유사어 추출
model2 = fasttext.load_model('noun_result.bin')
model2.get_nearest_neighbors("댕댕이")

[(0.0744122564792633, "'견사',"),
 (0.040044598281383514, "'애기',"),
 (0.03915612772107124, "'봉사',"),
 (0.03486886993050575, "'말티즈',"),
 (0.033787600696086884, "'가면',"),
 (0.033252838999032974, "'공간',"),
 (0.03272876888513565, "'이기',"),
 (0.029834795743227005, "'연신내',"),
 (0.02892761118710041, "'실제',"),
 (0.027488719671964645, "'다리',")]

## 자소 단위 학습

In [ ]:
#자소단위 텍스트 학습
save3 = 'naver_re_data.txt'
model_fname3 = 're_result.bin'

print("학습 중")
model3 = fasttext.train_unsupervised(
    input=save3,
    model='cbow',
    loss='hs',
    ws=11,
    lr=0.0001,
    dim=10,
    epoch=1000,
    min_count=5,
    thread=12,
    verbose=1,
)

model3.save_model(model_fname3)
print('완료')

학습 중


In [26]:
from soynlp.hangle import compose, decompose, character_is_korean

#자소단위 학습 - 형태 유사어 추출
model3 = fasttext.load_model('re_result.bin')
model3.get_nearest_neighbors("댕댕이")

[(0.9806052446365356, 'ㅊㅐㅇᴥㄱㅣㄹᴥ'),
 (0.9792996048927307, 'ㅈㅓㅇᴥㅌㅗㅇᴥㅇㅢᴥ'),
 (0.9792307019233704, 'ㅇㅢᴥ'),
 (0.9751275181770325, 'ㅁㅏᴥㅇㅡㅁᴥㅇㅢᴥ'),
 (0.9741119742393494, 'ㅈㅓㄴᴥㅌㅗㅇᴥㄱㅘᴥ'),
 (0.9738455414772034, 'ㅂㅡᴥㄹㅐㄴᴥㄷㅡᴥ'),
 (0.9699309468269348, 'ㅅㅏᴥㄹㅏㅇᴥㄱㅘᴥ'),
 (0.9669359922409058, 'ㅇㅕㅇᴥㅅㅏㅇᴥㅇㅢᴥ'),
 (0.9663962721824646, 'ㅇㅑㅇᴥㅇㅢᴥ'),
 (0.9659558534622192, 'ㅇㅟᴥㅎㅐᴥ')]

In [27]:
[hgtk.text.compose(found) for found in [word for score, word in model3.get_nearest_neighbors('댕댕이')]]

['챙길', '정통의', '의', '마음의', '전통과', '브랜드', '사랑과', '영상의', '양의', '위해']

# 의미 분석

In [1]:
from gensim.models import FastText
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fasttext

corpus_fname = 'naver_data.txt'
model_fname = 'fasttext'


print('corpus 생성')
corpus = [sent.strip().split(" ")for sent in tqdm(open(corpus_fname, 'r', encoding='utf-8').readlines())]

print("학습 중")
model = FastText(corpus, vector_size=100, window=5, min_count=5, workers=4, sg=1)
model.save(model_fname)
print('완료')

loaded_model = FastText.load("fasttext")
print(model.wv.vectors.shape)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
100%|█████████████████████████████████████████████████████████████████████████████| 104/104 [00:00<00:00, 26068.70it/s]

corpus 생성
학습 중


완료
(552, 100)


In [2]:
print(loaded_model.wv.most_similar("댕댕이", topn=10))

[('댕댕이삼계탕', 0.9992862343788147), ('강아지삼계탕', 0.9992821216583252), ('강아지옷', 0.9992791414260864), ('퓨어메라', 0.9992114901542664), ('이벤트', 0.9991117119789124), ('강아지간식', 0.999061107635498), ('강아지', 0.9990420937538147), ('강아지옷만들기', 0.9990110397338867), ('중입니다.분', 0.9989457130432129), ('애견용품', 0.998928427696228)]
